# Notebook de Preprocessamento

# Pré processamento dos dados

## Download da base de dados da PBH: 

Nesta sessão você vai baixar a base de dados da PBH e salvar em um arquivo CSV.

você pode fazer manualmente ou via código:



### Download manual

- [Link dos dados](https://ckan.pbh.gov.br/dataset/ec3efaac-0ca6-4846-9e32-0ffff2d76dbb/resource/4189b28e-7592-474b-8160-e39731315c27/download/20250401_atividade_economica.csv)
- baixar o csv e colocar na pasta `./data` com o nome de  `20250401_atividade_economica.csv`

### Download via código

Execute a próxima célula para baixar o arquivo CSV diretamente do CKAN da PBH e salvá-lo na pasta `./data`, com o nome de `20250401_atividade_economica.csv`.

In [1]:
!wget --user-agent="Mozilla/5.0" -O "./data/20250401_atividade_economica.csv" "https://ckan.pbh.gov.br/dataset/ec3efaac-0ca6-4846-9e32-0ffff2d76dbb/resource/4189b28e-7592-474b-8160-e39731315c27/download/20250401_atividade_economica.csv"

--2025-06-08 17:28:38--  https://ckan.pbh.gov.br/dataset/ec3efaac-0ca6-4846-9e32-0ffff2d76dbb/resource/4189b28e-7592-474b-8160-e39731315c27/download/20250401_atividade_economica.csv
Resolving ckan.pbh.gov.br (ckan.pbh.gov.br)... 170.82.174.5, 170.82.173.5
Connecting to ckan.pbh.gov.br (ckan.pbh.gov.br)|170.82.174.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 184789294 (176M) [text/csv]
Saving to: ‘./data/20250401_atividade_economica.csv’

./data/20250401_ati 100%[===================>] 176.23M  29.3MB/s    in 10s     

2025-06-08 17:28:49 (17.6 MB/s) - ‘./data/20250401_atividade_economica.csv’ saved [184789294/184789294]



## Import das Bibliotecas 


Para executar o notebook execute os imports das bibliotecas na célula abaixo.

Caso alguma não esteja instalada, execute o comando `pip install -R requirements.txt`,  assim você irá instalar todas as dependências necessárias.

In [2]:
from tqdm import tqdm
import pandas as pd
import re


from lib.utils import salvar_pkl
from lib.utils import normalize_name
from lib.utils import ler_pkl
from lib.geoinfo import geocode_single_address

## Leutura dos dados 

In [3]:
file_path = 'data/20250401_atividade_economica.csv'
df = pd.read_csv(file_path, sep=';')
df.head()

,ID_ATIV_ECON_ESTABELECIMENTO,CNAE_PRINCIPAL,DESCRICAO_CNAE_PRINCIPAL,CNAE,DATA_INICIO_ATIVIDADE,NATUREZA_JURIDICA,PORTE_EMPRESA,AREA_UTILIZADA,IND_SIMPLES,IND_MEI,...,FORMA_ATUACAO,DESC_LOGRADOURO,NOME_LOGRADOURO,NUMERO_IMOVEL,COMPLEMENTO,NOME_BAIRRO,NOME,NOME_FANTASIA,CNPJ,GEOMETRIA
0,1006,9521500.0,REPARAÇÃO E MANUTENÇÃO DE EQUIPAMENTOS ELETROE...,"3314707, 4322302, 4649404, 4651601, 4669999, 4...",19-04-1994,SOCIEDADE EMPRESÁRIA LIMITADA,EMPRESA DE PEQUENO PORTE,407.0,S,N,...,ESTABELECIMENTO FIXO,RUA,NIQUELINA,874,"GALPAO,",SANTA EFIGENIA,ORLANDO SERVICE LTDA,NaN,97502553000184,POINT (613854.46 7797085.18)
1,1007,4755502.0,COMERCIO VAREJISTA DE ARTIGOS DE ARMARINHO,"4755502, 8599604",12-04-1993,SOCIEDADE EMPRESÁRIA LIMITADA,MICROEMPRESA - ME,20.0,S,N,...,ESTABELECIMENTO FIXO,RUA,DOS GUAJAJARAS,910,SALA 612,CENTRO,TARGET LTDA,NaN,70950258000134,POINT (610658.10 7796519.70)
2,1008,4520001.0,SERVIÇOS DE MANUTENÇÃO E REPARAÇÃO MECÂNICA DE...,"4520001, 4520003, 4530703",16-11-1993,SOCIEDADE EMPRESÁRIA LIMITADA,MICROEMPRESA - ME,260.0,S,N,...,ESTABELECIMENTO FIXO,RUA,CONSELHEIRO LAFAIETE,284,NaN,SAGRADA FAMILIA,HUDSON CAR LTDA,HUDSON CAR,731235000153,POINT (612523.34 7798131.15)
3,1009,4616800.0,REPRESENTANTES COMERCIAIS E AGENTES DO COMÉRCI...,4616800,16-11-1992,SOCIEDADE EMPRESÁRIA LIMITADA,EMPRESA DE PEQUENO PORTE,1300.0,S,N,...,ESTABELECIMENTO FIXO,RUA,DOS TIMBIRAS,2352,SALA 01,LOURDES,SILHUETA REPRESENTACAO DE ROUPAS LTDA,GARMENT,68519362000127,POINT (610394.12 7796444.35)
4,1010,5822101.0,EDIÇÃO INTEGRADA À IMPRESSÃO DE JORNAIS DIÁRIOS,5822101,01-06-1993,SOCIEDADE EMPRESÁRIA LIMITADA,MICROEMPRESA - ME,30.0,N,N,...,ESTABELECIMENTO FIXO,RUA,SAO FELICIANO,73,NaN,CAICARAS,JORNAL CAICARAS LTDA,NaN,71078968000189,POINT (607875.47 7799877.16)


### Tratando prefixos dos Logradouros 

In [4]:
df['DESC_LOGRADOURO'].value_counts()

DESC_LOGRADOURO
RUA    447131
AVE     79646
BEC      2648
PCA      2053
ROD      1854
ALA      1007
EST       296
TRV       214
VIA        18
VDP         8
LRG         3
TRE         2
Name: count, dtype: int64

In [5]:
NOVO_LOGRADOURO = {'RUA': 'RUA', 'AVE': 'AV', 'PCA': 'PC', 'BEC': 'BC', 'ROD': 'RD', 'ALA': 'AL',
                   'EST': 'ESTRADA', 'TRV': 'TRV', 'VIA': 'VIA', 'VDP': 'V', 'LRG': 'LRG', 'TRE': 'TRE'}
# no dataset filtrado nao foi encontrado LRG e nem TRE, mative o mesmo

df['DESC_LOGRADOURO'] = df['DESC_LOGRADOURO'].map(NOVO_LOGRADOURO)
df

,ID_ATIV_ECON_ESTABELECIMENTO,CNAE_PRINCIPAL,DESCRICAO_CNAE_PRINCIPAL,CNAE,DATA_INICIO_ATIVIDADE,NATUREZA_JURIDICA,PORTE_EMPRESA,AREA_UTILIZADA,IND_SIMPLES,IND_MEI,...,FORMA_ATUACAO,DESC_LOGRADOURO,NOME_LOGRADOURO,NUMERO_IMOVEL,COMPLEMENTO,NOME_BAIRRO,NOME,NOME_FANTASIA,CNPJ,GEOMETRIA
0,1006,9521500.0,REPARAÇÃO E MANUTENÇÃO DE EQUIPAMENTOS ELETROE...,"3314707, 4322302, 4649404, 4651601, 4669999, 4...",19-04-1994,SOCIEDADE EMPRESÁRIA LIMITADA,EMPRESA DE PEQUENO PORTE,407.0,S,N,...,ESTABELECIMENTO FIXO,RUA,NIQUELINA,874,"GALPAO,",SANTA EFIGENIA,ORLANDO SERVICE LTDA,NaN,97502553000184,POINT (613854.46 7797085.18)
1,1007,4755502.0,COMERCIO VAREJISTA DE ARTIGOS DE ARMARINHO,"4755502, 8599604",12-04-1993,SOCIEDADE EMPRESÁRIA LIMITADA,MICROEMPRESA - ME,20.0,S,N,...,ESTABELECIMENTO FIXO,RUA,DOS GUAJAJARAS,910,SALA 612,CENTRO,TARGET LTDA,NaN,70950258000134,POINT (610658.10 7796519.70)
2,1008,4520001.0,SERVIÇOS DE MANUTENÇÃO E REPARAÇÃO MECÂNICA DE...,"4520001, 4520003, 4530703",16-11-1993,SOCIEDADE EMPRESÁRIA LIMITADA,MICROEMPRESA - ME,260.0,S,N,...,ESTABELECIMENTO FIXO,RUA,CONSELHEIRO LAFAIETE,284,NaN,SAGRADA FAMILIA,HUDSON CAR LTDA,HUDSON CAR,731235000153,POINT (612523.34 7798131.15)
3,1009,4616800.0,REPRESENTANTES COMERCIAIS E AGENTES DO COMÉRCI...,4616800,16-11-1992,SOCIEDADE EMPRESÁRIA LIMITADA,EMPRESA DE PEQUENO PORTE,1300.0,S,N,...,ESTABELECIMENTO FIXO,RUA,DOS TIMBIRAS,2352,SALA 01,LOURDES,SILHUETA REPRESENTACAO DE ROUPAS LTDA,GARMENT,68519362000127,POINT (610394.12 7796444.35)
4,1010,5822101.0,EDIÇÃO INTEGRADA À IMPRESSÃO DE JORNAIS DIÁRIOS,5822101,01-06-1993,SOCIEDADE EMPRESÁRIA LIMITADA,MICROEMPRESA - ME,30.0,N,N,...,ESTABELECIMENTO FIXO,RUA,SAO FELICIANO,73,NaN,CAICARAS,JORNAL CAICARAS LTDA,NaN,71078968000189,POINT (607875.47 7799877.16)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534875,460130,5229099.0,OUTRAS ATIVIDADES AUXILIARES DOS TRANSPORTES T...,5229099,08-05-2024,EMPRESÁRIO (INDIVIDUAL),MICROEMPRESA - ME,10.0,S,S,...,"PORTA A PORTA, POSTOS MÓVEIS OU POR AMBULANTES",RUA,ONZE DE NOVEMBRO,92,NaN,ALPES,55.045.831 FELIPE MIRANDA ALVES,NaN,55045831000171,POINT (608364.20 7793180.59)
534876,460131,6201501.0,DESENVOLVIMENTO DE PROGRAMAS DE COMPUTADOR SOB...,"6201501, 6202300, 6203100, 6204000",08-05-2024,SOCIEDADE EMPRESÁRIA LIMITADA,MICROEMPRESA - ME,20.0,S,N,...,ATIVIDADES DESENVOLVIDAS FORA DO ESTABELECIMEN...,RUA,RIO GRANDE DO NORTE,1435,"SALA: 708 ,PAVMTO: 7",SAVASSI,MARCOS R F DOS SANTOS LTDA,NaN,55046071000117,POINT (611699.07 7795022.80)
534877,460132,4761001.0,COMÉRCIO VAREJISTA DE LIVROS,"4761001, 5811500",08-05-2024,SOCIEDADE EMPRESÁRIA LIMITADA,DEMAIS,33.0,N,N,...,NaN,AV,CORONEL JOSE DIAS BICALHO,250,LOJA: 06,SAO LUIZ,WB2 - NEGOCIOS DIGITAIS LTDA,NaN,55046127000133,POINT (608711.71 7803934.64)
534878,460133,8599699.0,OUTRAS ATIVIDADES DE ENSINO NÃO ESPECIFICADAS ...,"4781400, 8599699",08-05-2024,EMPRESÁRIO (INDIVIDUAL),MICROEMPRESA - ME,10.0,S,S,...,"ESTABELECIMENTO FIXO, INTERNET, TELEVENDAS",AV,OIAPOQUE,156,ANEXO 2,CENTRO,55.045.904 CAMILA VITORIA PEREIRA DE ALMEIDA,NaN,55045904000125,POINT (611008.61 7797737.66)


### Filtragem por bares e restaurantes

In [6]:
words = ['bar', 'bares', 'restaurante', 'restaurantes']

Constrói a expressão regex que da match em qualquer uma das palavras, se ela aparecer como uma palavra separada por espaços ou pontuações das outras:

In [7]:
pattern = r'\b(?:' + '|'.join(re.escape(word) for word in words) + r')\b'
pattern

'\\b(?:bar|bares|restaurante|restaurantes)\\b'

In [8]:
filtered_table = df[['CNAE_PRINCIPAL', 'DESCRICAO_CNAE_PRINCIPAL']].drop_duplicates()
filtered_table = filtered_table[filtered_table['DESCRICAO_CNAE_PRINCIPAL'].str.contains(pattern, case=False, na=False)]
for value in filtered_table['DESCRICAO_CNAE_PRINCIPAL']:
    print(value)
filtered_table

RESTAURANTES E SIMILARES
BARES E OUTROS ESTABELECIMENTOS ESPECIALIZADOS EM SERVIR BEBIDAS, SEM ENTRETENIMENTO
BARES E OUTROS ESTABELECIMENTOS ESPECIALIZADOS EM SERVIR BEBIDAS, COM ENTRETENIMENTO 


,CNAE_PRINCIPAL,DESCRICAO_CNAE_PRINCIPAL
17,5611201.0,RESTAURANTES E SIMILARES
679,5611204.0,BARES E OUTROS ESTABELECIMENTOS ESPECIALIZADOS...
3079,5611205.0,BARES E OUTROS ESTABELECIMENTOS ESPECIALIZADOS...


In [9]:
CNAES = [int(cnae) for cnae in filtered_table['CNAE_PRINCIPAL']]
CNAES

[5611201, 5611204, 5611205]

In [10]:
filtered_df = df[df['CNAE_PRINCIPAL'].isin(CNAES)].copy()
filtered_df['CNAE_PRINCIPAL'].value_counts()

CNAE_PRINCIPAL
5611201.0    8550
5611204.0    3570
5611205.0    1505
Name: count, dtype: int64

### Formatação do Endereço

In [11]:
filtered_df['ENDERECO_FORMATADO'] = (
  filtered_df['DESC_LOGRADOURO'] + ' ' +
  filtered_df['NOME_LOGRADOURO'] + ', ' +
  filtered_df['NUMERO_IMOVEL'].astype(str)
)

filtered_df[['ENDERECO_FORMATADO']].head()

,ENDERECO_FORMATADO
17,"RUA DESEMBARGADOR REIS ALVES, 90"
19,"RUA LUIZ PONGELUPE, 290"
29,"AV RESSACA, 118"
172,"RUA DESEMBARGADOR RIBEIRO DA LUZ, 135"
363,"AV CRISTIANO MACHADO, 1950"


## Tabela Final

In [12]:
filtered_table = filtered_df[['DATA_INICIO_ATIVIDADE', 'IND_POSSUI_ALVARA', 'ENDERECO_FORMATADO', 'NOME_FANTASIA']].drop_duplicates()
filtered_table = filtered_table[filtered_table['NOME_FANTASIA'].notna()]
filtered_table['ID'] = range(1, filtered_table.shape[0]+1)
filtered_table.head()

,DATA_INICIO_ATIVIDADE,IND_POSSUI_ALVARA,ENDERECO_FORMATADO,NOME_FANTASIA,ID
19,15-10-1993,NÃO,"RUA LUIZ PONGELUPE, 290",PIZZARIA E CHURRASCARIA VARANDA,1
29,02-05-1994,NÃO,"AV RESSACA, 118",RABBIT BURGER,2
172,24-11-1993,SIM,"RUA DESEMBARGADOR RIBEIRO DA LUZ, 135",TATU REI DO ANGU A BAHIANA,3
363,01-10-1993,SIM,"AV CRISTIANO MACHADO, 1950",ESPACO DA FEIJOADA,4
662,16-09-2011,SIM,"RUA DOS OTONI, 222",RESTAURANTE NOVO SABOR,5


In [13]:
filtered_table.to_csv('data/dados_pre_tratados.csv', index=False)

## Coleta da Coordenada Geográfica

### Script para coletar dados do OpenStreetMap via Nominatim

In [ ]:
dados_geoloc = []
dados_with_error = []
for id, endereco in tqdm(filtered_table[['ID', 'ENDERECO_FORMATADO']].to_numpy()):
    latitude, longitude, EnderecoNominatim = geocode_single_address(endereco)
    if EnderecoNominatim == None: 
        dados_with_error.append([id, EnderecoNominatim])
    dados_geoloc.append([id, latitude, longitude, EnderecoNominatim])

  1%|          | 86/8657 [01:59<3:16:49,  1.38s/it]

Endereço não encontrado por Nominatim: AV ANTONIO HENRIQUE ALVES, 551, BELO HORIZONTE, MG


  1%|          | 102/8657 [02:24<4:50:17,  2.04s/it]

Endereço não encontrado por Nominatim: RUA BENJAMIM QUADROS, 266, BELO HORIZONTE, MG


  1%|          | 106/8657 [02:30<3:38:52,  1.54s/it]

...

 45%|████▍     | 3858/8657 [1:36:23<12:12:56,  9.16s/it]

 45%|████▍     | 3859/8657 [1:36:33<12:33:07,  9.42s/it]

 47%|████▋     | 4038/8657 [1:36:42<06:35, 11.67it/s]   

 56%|█████▌    | 4842/8657 [1:36:42<00:20, 183.68it/s]

100%|██████████| 8657/8657 [1:37:57<00:00,  1.47it/s] 

### Salvando os dados geolocalizados para casos de falha

In [ ]:
salvar_pkl(dados_geoloc, './temp/dado_geoloc_sucesso.pkl')

Arquivo salvo em: ./temp/dado_geoloc_sucesso.pkl


## Juntando os dados Geolocalizados com os dados da PBH Tratados

In [14]:
dados_geoloc = ler_pkl( './temp/dado_geoloc_sucesso.pkl')
df_dados_geoloc = pd.DataFrame(dados_geoloc, columns=['ID', 'LATITUDE', 'LONGITUDE', 'ENDERECO_NOMINATIM'])
df_dados_geoloc['ID']= df_dados_geoloc['ID'].astype(int)
df_dados_geoloc  = df_dados_geoloc.sort_values(by='ID')  
df_dados_geoloc

,ID,LATITUDE,LONGITUDE,ENDERECO_NOMINATIM
4604,1,-19.799323,-43.976029,"Rua Luiz Pongellupe, Vila SESC, Jardim dos Com..."
4605,2,-19.921143,-43.986082,"Avenida Ressaca, Coração Eucarístico, Regional..."
4606,3,-19.973201,-44.014192,"Rua Desembargador Ribeiro da Luz, Barreiro de ..."
4607,4,-19.890369,-43.928219,"Feira dos Produtores, 1950, Avenida Cristiano ..."
4608,5,-19.926088,-43.926185,"Rua dos Otoni, Santa Efigênia, Regional Centro..."
...,...,...,...,...
3757,8653,-19.868581,-43.987023,"Rua Marcos de Oliveira, Ouro Preto, Itamarati,..."
3758,8654,-19.922520,-43.919660,"Avenida do Contorno, São Lucas, Santa Efigênia..."
3759,8655,-19.944486,-43.934504,"Avenida Nossa Senhora do Carmo, São Pedro, Reg..."
3760,8656,-19.876441,-44.002659,"Rua Castelo de Dover, Castelo, Pampulha, Belo ..."


In [15]:
df_full_dataset = pd.merge(filtered_table, df_dados_geoloc, on='ID', how='left')
df_full_dataset['HAS_GEOLOC'] = df_full_dataset['LATITUDE'].notna()
df_full_dataset

,DATA_INICIO_ATIVIDADE,IND_POSSUI_ALVARA,ENDERECO_FORMATADO,NOME_FANTASIA,ID,LATITUDE,LONGITUDE,ENDERECO_NOMINATIM,HAS_GEOLOC
0,15-10-1993,NÃO,"RUA LUIZ PONGELUPE, 290",PIZZARIA E CHURRASCARIA VARANDA,1,-19.799323,-43.976029,"Rua Luiz Pongellupe, Vila SESC, Jardim dos Com...",True
1,02-05-1994,NÃO,"AV RESSACA, 118",RABBIT BURGER,2,-19.921143,-43.986082,"Avenida Ressaca, Coração Eucarístico, Regional...",True
2,24-11-1993,SIM,"RUA DESEMBARGADOR RIBEIRO DA LUZ, 135",TATU REI DO ANGU A BAHIANA,3,-19.973201,-44.014192,"Rua Desembargador Ribeiro da Luz, Barreiro de ...",True
3,01-10-1993,SIM,"AV CRISTIANO MACHADO, 1950",ESPACO DA FEIJOADA,4,-19.890369,-43.928219,"Feira dos Produtores, 1950, Avenida Cristiano ...",True
4,16-09-2011,SIM,"RUA DOS OTONI, 222",RESTAURANTE NOVO SABOR,5,-19.926088,-43.926185,"Rua dos Otoni, Santa Efigênia, Regional Centro...",True
...,...,...,...,...,...,...,...,...,...
8652,24-04-2024,SIM,"RUA MARCOS DE OLIVEIRA, 317",PASSELI PIZZAS E PAES ARTESANAIS,8653,-19.868581,-43.987023,"Rua Marcos de Oliveira, Ouro Preto, Itamarati,...",True
8653,24-04-2024,SIM,"AV DO CONTORNO, 4347",ARMAZEM DO CHOPP CHOPERIA E PETISCARIA,8654,-19.922520,-43.919660,"Avenida do Contorno, São Lucas, Santa Efigênia...",True
8654,02-05-2024,NÃO,"AV NOSSA SENHORA DO CARMO, 683",BAR IRMAOS GONCALVES,8655,-19.944486,-43.934504,"Avenida Nossa Senhora do Carmo, São Pedro, Reg...",True
8655,17-04-2024,SIM,"RUA CASTELO DE DOVER, 70",GORDINHO CARNES E BEBIBAS BH,8656,-19.876441,-44.002659,"Rua Castelo de Dover, Castelo, Pampulha, Belo ...",True


## Juntando com dados de Comida de Boteco

Aqui vamos pegar os dados extraidos do site sobre o festival comida de boteco.

Para obter esses dados, execute o notebook [./extract_comida_di_boteco.ipynb](./extract_comida_di_boteco.ipynb)

In [18]:
df_comida = pd.read_csv('./data/comida_di_boteco_2025.csv').rename(columns={
    'bar_name': 'BAR_NAME',
    'plate_name': 'NOME_PRATO',
    'plate_description': 'PRATO_DESCRICAO',
    'endereco': 'ENDERECO_FORMATADO',
    'image_path': 'PRATO_IMAGEM',
})
df_comida['HAS_CDB'] = True
df_comida

,BAR_NAME,NOME_PRATO,PRATO_DESCRICAO,ENDERECO_FORMATADO,PRATO_IMAGEM,HAS_CDB
0,Alexandre’s Bar,O fabuloso discão,Polpetone com recheio de muçarela acompanhada ...,"Rua David Alves do Vale, 68 | Santa Rosa, Belo...",https://s2-g1.glbimg.com/w6xgTZb45ExVKcLoRAFIa...,True
1,Amarelim do Prado,Trem de roças,"Bochecha de porco na cama de angu mole, pesto ...","Avenida Francisco Sá, 658 | Prado, Belo Horizonte",https://s2-g1.glbimg.com/A1sXFeWp6YE12bpInykst...,True
2,Andrade’s Beer,Chora canelinha,"Ossobuco com purê de batatas, acompanhado de t...","Rua Dona Geni, 32 | Jardim Florência (Venda No...",https://s2-g1.glbimg.com/9tZo4O55GproBN850s0km...,True
3,Arcos Bar e Restaurante,Panela da Diversidade,"Acém cozido com ervas finas, acompanhado com b...","Rua da Bahia, 1144 | Centro, Belo Horizonte.",https://s2-g1.glbimg.com/TBJlO2gBA-eoaA_0XK6kM...,True
4,Armazém Santa Amélia,Segredo do Armazém,Polenta cremosa com ragu de carnes cheio de sa...,"Avenida da Sinfonia, 253 | Santa Amélia, Belo ...",https://s2-g1.glbimg.com/z23BBsunx09X3iGDgJq31...,True
...,...,...,...,...,...,...
116,Xambar,Mineirin Oriental,"Miolo de acém assado na pressão, acompanhado d...","Avenida Augusto de Lima, 233 | Centro, Belo Ho...",https://cdb-static-files.s3.amazonaws.com/wp-c...,True
117,Xico da Kafua,Pé na areia,"Mão suína, acompanhada de farofa crocante de s...","Avenida Itaú, 1195 – Loja A | João Pinheiro, B...",https://cdb-static-files.s3.amazonaws.com/wp-c...,True
118,Xico do Churrasco,Xico Balaio,Asinhas de frango com provolone com tempero.,"Av Altamiro avelino Soares 920 – Castelo, Belo...",https://cdb-static-files.s3.amazonaws.com/wp-c...,True
119,Zé Bolacha,Rabicó,Bolinho de costelinha desfiada com requeijão e...,"Rua das Melancias, 35 | Vila Cloris, Belo Hori...",https://comidadibuteco.com.br/wp-content/uploa...,True


In [19]:
df_comida['NORMALIZED_NAME'] = df_comida['BAR_NAME'].apply(normalize_name)
df_full_dataset['NORMALIZED_NAME'] = df_full_dataset['NOME_FANTASIA'].apply(normalize_name)

In [24]:
df_full_dataset_complete = pd.merge(
    left=df_full_dataset,
    right=df_comida[['NORMALIZED_NAME', 'NOME_PRATO', 'PRATO_DESCRICAO', 'PRATO_IMAGEM','HAS_CDB']],
    how='left',
    on='NORMALIZED_NAME',
)
df_full_dataset_complete['HAS_CDB'] = df_full_dataset_complete['HAS_CDB'].fillna(False).astype(bool)
df_full_dataset_complete

/var/folders/33/gy7xm05x097chg59653_qr3m0000gn/T/ipykernel_37428/2775903342.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_full_dataset_complete['HAS_CDB'] = df_full_dataset_complete['HAS_CDB'].fillna(False).astype(bool)


,DATA_INICIO_ATIVIDADE,IND_POSSUI_ALVARA,ENDERECO_FORMATADO,NOME_FANTASIA,ID,LATITUDE,LONGITUDE,ENDERECO_NOMINATIM,HAS_GEOLOC,NORMALIZED_NAME,NOME_PRATO,PRATO_DESCRICAO,PRATO_IMAGEM,HAS_CDB
0,15-10-1993,NÃO,"RUA LUIZ PONGELUPE, 290",PIZZARIA E CHURRASCARIA VARANDA,1,-19.799323,-43.976029,"Rua Luiz Pongellupe, Vila SESC, Jardim dos Com...",True,pizzaria_e_churrascaria_varanda,NaN,NaN,NaN,False
1,02-05-1994,NÃO,"AV RESSACA, 118",RABBIT BURGER,2,-19.921143,-43.986082,"Avenida Ressaca, Coração Eucarístico, Regional...",True,rabbit_burger,NaN,NaN,NaN,False
2,24-11-1993,SIM,"RUA DESEMBARGADOR RIBEIRO DA LUZ, 135",TATU REI DO ANGU A BAHIANA,3,-19.973201,-44.014192,"Rua Desembargador Ribeiro da Luz, Barreiro de ...",True,tatu_rei_do_angu_a_bahiana,NaN,NaN,NaN,False
3,01-10-1993,SIM,"AV CRISTIANO MACHADO, 1950",ESPACO DA FEIJOADA,4,-19.890369,-43.928219,"Feira dos Produtores, 1950, Avenida Cristiano ...",True,espaco_da_feijoada,NaN,NaN,NaN,False
4,16-09-2011,SIM,"RUA DOS OTONI, 222",RESTAURANTE NOVO SABOR,5,-19.926088,-43.926185,"Rua dos Otoni, Santa Efigênia, Regional Centro...",True,restaurante_novo_sabor,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8652,24-04-2024,SIM,"RUA MARCOS DE OLIVEIRA, 317",PASSELI PIZZAS E PAES ARTESANAIS,8653,-19.868581,-43.987023,"Rua Marcos de Oliveira, Ouro Preto, Itamarati,...",True,passeli_pizzas_e_paes_artesanais,NaN,NaN,NaN,False
8653,24-04-2024,SIM,"AV DO CONTORNO, 4347",ARMAZEM DO CHOPP CHOPERIA E PETISCARIA,8654,-19.922520,-43.919660,"Avenida do Contorno, São Lucas, Santa Efigênia...",True,armazem_do_chopp_choperia_e_petiscaria,NaN,NaN,NaN,False
8654,02-05-2024,NÃO,"AV NOSSA SENHORA DO CARMO, 683",BAR IRMAOS GONCALVES,8655,-19.944486,-43.934504,"Avenida Nossa Senhora do Carmo, São Pedro, Reg...",True,bar_irmaos_goncalves,NaN,NaN,NaN,False
8655,17-04-2024,SIM,"RUA CASTELO DE DOVER, 70",GORDINHO CARNES E BEBIBAS BH,8656,-19.876441,-44.002659,"Rua Castelo de Dover, Castelo, Pampulha, Belo ...",True,gordinho_carnes_e_bebibas_bh,NaN,NaN,NaN,False


In [27]:
df_full_dataset_complete['HAS_CDB'].value_counts()

HAS_CDB
False    8562
True       95
Name: count, dtype: int64

## Salvando os Dados Processados

In [26]:
df_full_dataset_complete.to_csv('data/dados_tratados_geolocalizados.csv', index=False)